# Covered here

At a broad level, this walkthrough covers three things:
1. Using Python's `timeit` module in the interpreter;
2. Calling Python's `timeit` from the command line;
3. IPython's `%time` and `%timeit` magic functions.

Contents:
- [`timeit` in the interactive interpreter](#timeit-in-the-interpreter)
    - [Refresher: units of time](#Refresher:-units-of-time)
    - [Intro to the `timeit` module](#Intro)
    - [`number` versus `repeat` parameters](#number-versus-repeat-parameters)
    - [The `timeit.Timer` class](#timeit.Timer)
    - [`timeit.timeit()`](#timeit.timeit)
    - [`timeit.repeat()`](#timeit.repeat)
    - [`timeit.default_timer()`](#timeit.default_timer)
    - [Using `from __main__ import ...`](#Using-from-__main__-import-...)
    - [Why is `timeit.timeit()` better than `time.time()`?](#Why-is-timeit.timeit-better-than-time.time?)
- [`timeit`: command-line interface](#timeit:-command-line-interface)
    - [Timing code snippets](#Timing-code-snippets)
    - [Timing scripts](#Timing-scripts)
- [Ipython/Jupyter magic functions](#IPython/Jupyter-magic-functions)
    - [`%time`](#%time)
    - [`%timeit`](#%timeit)
    - [Aliasing a custom `%timeit`](#Aliasing-a-custom-%timeit)

# Resources & references

- Python 3 docs: [`timeit`](https://docs.python.org/3/library/timeit.html) module
    - The module source: [cpython/Lib/timeit.py](https://github.com/python/cpython/blob/3.6/Lib/timeit.py)
    - The [command-line interface](https://docs.python.org/3/library/timeit.html#command-line-interface); command-line main function [source](https://github.com/python/cpython/blob/3.6/Lib/timeit.py#L240)
    - [Examples](https://docs.python.org/3/library/timeit.html#examples)
- mail.python.org: [python: can't open file 'timeit.py'](https://mail.python.org/pipermail/python-list/2005-January/345201.html)
- IPython magics: [`%time`](http://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-time) and [`%timeit`](http://ipython.readthedocs.io/en/stable/interactive/magics.html#magic-timeit)
- Stack Overflow:
    - [time.time vs. timeit.timeit](https://stackoverflow.com/questions/17579357/time-time-vs-timeit-timeit)
    - [How to use `timeit` module](https://stackoverflow.com/questions/8220801/how-to-use-timeit-module)
- Hellman - _The Python Standard Library by Example_ - Chapter 16.9 (`timeit`)

# `timeit` in the interpreter

## Refresher: units of time

| Time | Relative |
| :--- | :------- |
| Millisecond | 1/1000 of a second |
| Microsecond | 1/1,000,000 of a second |
| Nanosecond | One billionth of a second |

`timeit.timeit` and `%timeit`/`%time` use **seconds, microseconds (ms), or nanoseconds ($\mu$s).**

The docs for `timeit.timeit` have a `-u` parameter/flag:

```python
-u, --unit=U
    specify a time unit for timer output; can select usec, msec, or sec
```

`msec` is millisecond, not microsecond; proof:

In [1]:
%%bash
python3 -m timeit -n 1000 -r 100 -u usec '"-".join(str(n) for n in range(100))'

1000 loops, best of 100: 21.8 usec per loop


In [2]:
%%bash
python3 -m timeit -n 1000 -r 100 -u msec  '"-".join(str(n) for n in range(100))'

1000 loops, best of 100: 0.0222 msec per loop


In [3]:
%%bash
python3 -m timeit -n 1000 -r 100 -u sec  '"-".join(str(n) for n in range(100))'

1000 loops, best of 100: 2.21e-05 sec per loop


Above, `stmt` takes 21.4 nanoseconds (usec), equivalent to 0.0214 microseconds or 0.00002140 milliseconds.  So, `msec` is a microsecond here, as evidenced by the second run>.  (Also, [here](https://github.com/python/cpython/blob/3.6/Lib/timeit.py#L276) is where `units` is defined.)

## Intro

The `timeit` module provides a simple interface for determining the execution time of small bits of Python code. It uses a **platform-specific time function** to provide the most accurate time calculation possible and reduces the impact of start-up or shutdown costs on the time calculation by executing the code repeatedly.

There are 4 public objects in the `timeit` library (one class and three functions):

```python
__all__ = ["Timer", "timeit", "repeat", "default_timer"]
```

Here we will take a look at each of those 4:

In [4]:
# just for sake of never using "import *"...
from timeit import Timer, timeit, repeat, default_timer

Here's a simplified tree structure of the module.  Internally, **the functions `timeit.timeit` and `timeit.repeat` create instances of `Timer`, and then call the the class's corresponding methods.**

```python
import time

default_timer = time.perf_counter
default_number = 1000000
default_repeat = 3

# ...

class Timer:
    def timeit(self, number=default_number):
        # ...
    def repeat(self, repeat=default_repeat, number=default_number):
        # ...

# ...

def timeit(stmt="pass", setup="pass", timer=default_timer,
           number=default_number, globals=None):
    return Timer(stmt, setup, timer, globals).timeit(number)

def repeat(stmt="pass", setup="pass", timer=default_timer,
           repeat=default_repeat, number=default_number, globals=None):
    return Timer(stmt, setup, timer, globals).repeat(repeat, number)
```

## `number` versus `repeat` parameters

There are two ambiguous parameters involved within `timeit.py`, `number` and `repeat`:

```python
-n N, --number=N
    how many times to execute ‘statement’ [execute given statement N times in a loop]

-r N, --repeat=N
    how many times to repeat the loop iteration (default 3)
```

(Or, in `%timeit`, this is `%%timeit [-n<N> -r<R> ...`.)

**The number of repeats is used to determine the average (or, the best)** of the series of loops.

| Function/method | `number` default | `repeat` default |
| :-------------- | :--------------- | :--------------- |
| `timeit.main()` (command-line interface) | Variable.  A suitable number of loops is calculated by trying successive powers of 10 until the total time is at least 0.2 seconds. | 3 |
| `timeit.Timer.timeit()` | 1,000,000 | n/a |
| `timeit.Timer.repeat()` | 1,000,000 | 3 |
| `timeit.timeit()` | 1,000,000 | n/a |
| `timeit.repeat()` | 1,000,000 | 3 |
| `%timeit` | Variable.  A fitting value is chosen. | 7, although the docs say 3 [[source](https://github.com/ipython/ipython/blob/master/IPython/core/magics/execution.py#L1022)] |
| `%time` | n/a | n/a |

From the command line:

In [5]:
%%bash
# Defaults: -n is variable; -r is 3
python3 -m timeit '[i+1 for i in range(5000)]'

1000 loops, best of 3: 300 usec per loop


In [6]:
%%bash
# This statement gets executed 3 * 250 times (`r` has default 3)
python3 -m timeit -n 250 '[i+1 for i in range(5000)]'

250 loops, best of 3: 317 usec per loop


In [7]:
%%bash
# This statement gets executed 10 * 250 times (`r` specified explicitly)
python3 -m timeit -n 250 -r 10 '[i+1 for i in range(5000)]'

250 loops, best of 10: 316 usec per loop


With IPython magics:

In [8]:
# -n is variable; -r is 7
%timeit [i+1 for i in range(5000)]

303 µs ± 10.5 µs per loop (mean ± std. dev. of 7 runs, 1000 loops each)


In [9]:
%timeit -n 250 -r 50 [i+1 for i in range(5000)]

287 µs ± 19.4 µs per loop (mean ± std. dev. of 50 runs, 250 loops each)


## `timeit.Timer`

First instantiate a `Timer` object:

In [10]:
t = Timer(stmt="print('main statement')", setup="print('setup')")

Note that `Timer.timeit()` gives you **cumulative** (not minimum!) time to run _n_ number of times.

In [11]:
# *method* syntax: `timeit(number=1000000)`
print('TIMEIT:\n', t.timeit(number=2), sep='')

setup
main statement
main statement
TIMEIT:
5.988100019749254e-05


In [12]:
print('TIMEIT:\n', t.timeit(2), '\nREPEAT:\n', t.repeat(repeat=3, number=2), sep='')

setup
main statement
main statement
setup
main statement
main statement
setup
main statement
main statement
setup
main statement
main statement
TIMEIT:
0.0001329299993813038
REPEAT:
[6.382400169968605e-05, 5.2911000238964334e-05, 0.0003373710023879539]


## `timeit.timeit`

The syntax for `timeit.timeit` is:

> `timeit.timeit(stmt='pass', setup='pass', timer=<default timer>, number=1000000, globals=None)`

Two important points:
- `stmt` and `setup` may also contain:
    - multiple statements separated by ";", or
    - newlines, as long as they don’t contain multi-line string literals.
- The execution time of `setup` is excluded from the overall timed execution run.

In [13]:
timeit('char in text', setup='text = "sample string"; char = "g"')

0.05012419899867382

In [14]:
timeit('text.find(char)', setup='text = "sample string"; char = "g"')

0.15175113800069084

In [15]:
from tabulate import tabulate

lengths = np.power(2, np.arange(15))  # 1, 2, 4, 8, 16, 31, ...

list_time = []
dict_time = []
for length in lengths:
    list_time.append(timeit(stmt='%i in d' % (length/2), setup='d=range(%i)' % length))
    dict_time.append(timeit(stmt='%i in d' % (length/2),
                            setup='d=dict.fromkeys(range(%i))' % length))
print(tabulate(zip(*[lengths, list_time, dict_time]),
               headers=('i', 'list time', 'dict time')))

    i    list time    dict time
-----  -----------  -----------
    1    0.0842133    0.0331339
    2    0.0744037    0.035701
    4    0.0794738    0.0377766
    8    0.0804355    0.0313973
   16    0.072243     0.0301475
   32    0.0778933    0.028557
   64    0.0735351    0.0289966
  128    0.0767174    0.030355
  256    0.0782498    0.0312843
  512    0.0754935    0.0290933
 1024    0.0827858    0.0424235
 2048    0.0806586    0.0406065
 4096    0.0868734    0.0476545
 8192    0.0902657    0.0424241
16384    0.0823362    0.04333


In [16]:
setup = """
import msgpack
import json
from copy import deepcopy
data = {'name':'John Doe',
        'ranks':{'sports':13,'edu':34,'arts':45},
        'grade':5}"""

print(timeit('deepcopy(data)', setup=setup))
print(timeit('json.loads(json.dumps(data))', setup=setup))
print(timeit('msgpack.unpackb(msgpack.packb(data))', setup=setup))

9.604499187000329
7.8452203839988215
4.140860599000007


In [17]:
print(timeit("list_1[:][:]", 
             setup="list_1 = [[1, 2, 3], [4, 5, 6], [7, 8, 9]]"))

print(timeit("list_2[:,]", 
              setup="import numpy as np; list_2 = np.array([[1, 2, 3], [4, 5, 6], [7, 8, 9]])"))

0.18069548399944324
0.22837112200068077


In [18]:
import sys

# Initialize a list of (str, int) tuples 
# that the main `stmt`s will use to build dictionaries,
# using the strings as keys and storing the integers 
# as the associated values.

range_size = 1000
count = 1000
setup = "l = [(str(x), x) for x in range(1000)]; d = {}"

def show_results(result):
    """Utility func: print results in terms of ms/pass and ms/item."""
    global count, range_size
    per_pass = 1000000 * (result / count)
    print('%.2f usec/pass' % per_pass)
    per_item = per_pass / range_size
    print('%.2f usec/item' % per_item)
    
print("%d items" % range_size)
print("%d iterations" % count)

1000 items
1000 iterations


In [19]:
# Route 1 - Using `__setitem__` without checking 
# for existing values first
print('__setitem__:')
t = Timer("""
for s, i in l:
    d[s] = i
""", setup)
show_results(t.timeit(number=count))

__setitem__:
63.72 usec/pass
0.06 usec/item


In [20]:
# Route 2 - setdefault() to ensure that values already 
# in the dictionary are not overwritten.
print('setdefault:')
t = Timer("""
for s, i in l:
    d.setdefault(s, i)
""", setup)
show_results(t.timeit(number=count))

setdefault:
125.08 usec/pass
0.13 usec/item


In [21]:
# Method 3 - Another way to avoid overwriting existing 
# values is to use `in` (`__contains__`) to check the contents of 
# the dictionary explicitly.  (This was `haskey` in Python 2.x)
print('has_key:')
t = Timer("""
for s, i in l:
    if s not in d:
        d[s] = i
""", setup)
show_results(t.timeit(number=count))

has_key:
52.99 usec/pass
0.05 usec/item


## `timeit.repeat`

This is a convenience function that **calls the `timeit()` repeatedly, returning a list of results. **
- The `repeat` specifies how many times to call `timeit()`.
- The `number` argument specifies the number argument for `timeit()`.

Syntax for `timeit.repeat`:

> `timeit.repeat(stmt='pass', setup='pass', timer=<default timer>, repeat=3, number=1000000, globals=None)`

Notice this has a `repeat` parameter whereas `timeit.timeit` does not.

```python
class Timer:
    # ...
    def repeat(self, repeat=default_repeat, number=default_number):
        r = []
        for i in range(repeat):
            t = self.timeit(number)
            r.append(t)
        return r
```

In [22]:
def reverse_string(s):
    return s[::-1]

s = 'abcdefghijklmnopqrstuvwxyz' * 100
repeat(lambda: reverse_string(s))

[1.4181391519996396, 1.4050594389991602, 1.393595909998112]

In [23]:
setup = '''
import random

random.seed(123)
s = [random.random() for i in range(1000)]
timsort = list.sort
'''

print(min(Timer('a=s[:]; timsort(a)', setup=setup).repeat(7, 1000)))

0.1559878590014705


## `timeit.default_timer`

This function can be useful in situations where you want to run code and time that code chunk at simultaneously.  However, note that **this doesn't disable garbage collection or separate out system time.**

In [24]:
import pandas as pd

start = default_timer()
url = 'http://web.mta.info/developers/data/nyct/turnstile/turnstile_180106.txt'
column_types = {
    'DIVISION': 'category',
    'DESC': 'category',
    'LINENAME': 'category',
    'STATION': 'category',
    'C/A': 'category',
    }
df = pd.read_csv(url, parse_dates=[['DATE', 'TIME']], 
                 infer_datetime_format=True, dtype=column_types)
end = default_timer() - start
print('Retrieved {:,} records in {:0.2f} seconds.'.format(len(df), end))

Retrieved 200,665 records in 4.19 seconds.


## Using `from __main__ import ...`

If for whatever reason you aren't using `%timeit`, you may need a `setup` syntax like `from __main__ import func` if you've defined `func` in an interactive session and want to time it:

In [25]:
def f(x):
    return x*x - x/x**2

repeat("for x in range(1, 100): f(x)", setup="from __main__ import f", 
       number=100000)

[4.197107559997676, 4.200084685002366, 4.233881822998228]

## Why is `timeit.timeit` better than `time.time`?

From [Martijn Pieters](https://stackoverflow.com/a/17579466/7954504):

`timeit` is more accurate, for three reasons:

* it repeats the tests many times to eliminate the influence of other tasks on your machine, such as disk flushing and OS scheduling.
* it disables the [garbage collector](https://docs.python.org/3/glossary.html#term-garbage-collection) to prevent that process from skewing the results by scheduling a collection run at an inopportune moment.
* it picks the most accurate timer for your OS, `time.time` or `time.clock` in Python 2 and [`time.perf_counter()`](https://docs.python.org/3/library/time.html#time.perf_counter) on Python 3. See [`timeit.default_timer`](http://docs.python.org/3/library/timeit.html#timeit.default_timer).

# `timeit`: command-line interface

When called as a program from the command line, the following form is used:

```bash
python3 -m timeit [-n N] [-r N] [-u U] [-s S] [-t] [-c] [-h] [statement ...]
```

This executes `main()` within `timeit.py`, which:
1. [Calls `Timer.repeat()`](https://github.com/python/cpython/blob/3.6/Lib/timeit.py#L329);
2. [Calculates a `best` time](https://github.com/python/cpython/blob/3.6/Lib/timeit.py#L333) from these runs;
3. Prints the _average time per loop_ from the `best` run.

## Timing code snippets

Some examples are below.  (Here we use `%%bash` which is an IPython cell magic that runs cells with bash in a subprocess.)

Using the `-m` flag tells the Python interpreter to find the module and treat it as the main program.

The `stmt` argument works a little differently on the command line than the
argument to `Timer`: 
- Instead of one long string, pass each line of the instructions as a separate command-line argument. 
- To indent lines (such as inside a loop), embed spaces in the string by enclosing it in quotes.

From @Veedrac:
> On the command line, `timeit` does proper statistical analysis: it tells you how long the shortest run took. This is good because all error in timing is positive. So the shortest time has the least error in it. There's no way to get negative error because a computer can't ever compute faster than it can compute!

In [26]:
%%bash
python3 -m timeit '"-".join(str(n) for n in range(100))'

10000 loops, best of 3: 22.7 usec per loop


In [27]:
%%bash
python3 -m timeit '"-".join([str(n) for n in range(100)])'

10000 loops, best of 3: 20.5 usec per loop


In [28]:
%%bash
python3 -m timeit '"-".join(map(str, range(100)))'

100000 loops, best of 3: 15.7 usec per loop


In [29]:
%%bash
python3 -m timeit -s 'text = "sample string"; char = "g"'  'char in text'

10000000 loops, best of 3: 0.0304 usec per loop


In [30]:
%%bash
python3 -m timeit -s 'text = "sample string"; char = "g"'  'text.find(char)'

10000000 loops, best of 3: 0.136 usec per loop


Note that the [source](https://github.com/python/cpython/blob/3.6/Lib/timeit.py#L11) says:

> Command line usage:
> 
> `python timeit.py [-n N] [-r N] [-s S] [-t] [-c] [-p] [-h] [--] [statement]`

However, this is not really correct, unless you `cd` to `timeit.py`.  Alternatively, use the syntax above i.e. `python -m timeit ...`.  A discussion about this is [here](https://mail.python.org/pipermail/python-list/2005-January/345201.html).

Note that you can use repeated flags:

In [31]:
%%bash
python3 -m timeit -s "x = range(10000)" -s "y = range(100)" "sum(x)" "min(y)"

10000 loops, best of 3: 156 usec per loop


The above has `setup`:

```python
x = range(1000)
y = range(100)
```

and two statements:

```python
sum(x)
min(y)
```

## Timing scripts

Say we have the following `.py` file located in _/Users/brad/Scripts/python/docs/tutorials/imgs/_:

```python
# timeit_setitem.py
def test_setitem(range_size=1000):
    q = [(str(x), x) for x in range(range_size)]
    d = {}
    for s, i in q:
        d[s] = i
```

We can time its function from the command line like so:

In [32]:
%%bash
cd /Users/brad/Scripts/python/docs/tutorials/imgs/
python3 -m timeit -s 'import timeit_setitem' 'timeit_setitem.test_setitem()'

1000 loops, best of 3: 286 usec per loop


# IPython/Jupyter magic functions

## `%time`

From the docs: 

> This function provides very basic timing functionality. **Use the `timeit` magic for more control over the measurement.**

`%time` can be called two ways:

In **cell mode**, you can time the cell body (a directly following statement raises an error).

In [33]:
%time  # Can't have anything here though...
a = "-".join(str(n) for n in range(100))
print(a[:10], '...')

CPU times: user 5 µs, sys: 2 µs, total: 7 µs
Wall time: 11.2 µs
0-1-2-3-4- ...


In **line mode** you can time a **single-line statement** (though multiple ones can be chained with using semicolons):

In [34]:
%time a = "-".join(str(n) for n in range(100))

CPU times: user 50 µs, sys: 1 µs, total: 51 µs
Wall time: 57 µs


## `%timeit`

Similar to `%time`, you can use [`%timeit`](https://github.com/ipython/ipython/blob/master/IPython/core/magics/execution.py#L945) in line mode or cell mode:

Line mode usage:

> `%timeit [-n<N> -r<R> [-t|-c] -q -p<P> -o] statement`

Cell mode uisage:

> `%%timeit [-n<N> -r<R> [-t|-c] -q -p<P> -o] setup_code code code...`

In [35]:
%%timeit
mult_of_six = []  # The first line is setup code
for i in range(100000):
    if i % 2 == 0 and i % 3 == 0:
        mult_of_six.append(i)

9.11 ms ± 399 µs per loop (mean ± std. dev. of 7 runs, 100 loops each)


We can use the `-o` flag to return a [`TimeitResult`](https://github.com/ipython/ipython/blob/master/IPython/core/magics/execution.py#L56) object that can be stored in a variable to inspect the result in more details.  The below also uses `-q` (quiet: don't print the result).

Note that `%timeit` does **not** have a `-u` flag.

In [36]:
def f():
    for i in range(100000):
        if i % 2 == 0 and i % 3 == 0:
            yield i
    return list(mult_of_six)
    
res = %timeit -oq -n 500 -r 7 f()

In [37]:
for attr in dir(res):
    if not attr.startswith('_'):
        disp = getattr(res, attr)
        if isinstance(disp, int):
            print(attr, ': ', getattr(res, attr), sep='')
        elif isinstance(disp, float):
            print(attr, ': ', '{:0.2g}'.format(getattr(res, attr)), sep='')
        elif isinstance(disp, list):
            print(attr, ': ', ['{:0.2g}'.format(i) 
                               for i in getattr(res, attr)], sep='')

all_runs: ['9.3e-05', '9.2e-05', '9.2e-05', '9.2e-05', '9.1e-05', '9.2e-05', '9.2e-05']
average: 1.8e-07
best: 1.8e-07
compile_time: 5.7e-05
loops: 500
repeat: 7
stdev: 7.3e-10
timings: ['1.9e-07', '1.8e-07', '1.8e-07', '1.8e-07', '1.8e-07', '1.8e-07', '1.8e-07']
worst: 1.9e-07


## Aliasing a custom `%timeit`

You can alias `%timeit -oq -n 500 -r 7` by placing this in your `startup.py` file (on Mac, `~/.ipython/profile_default/startup/startup.py`):

```python

# from IPython import get_ipython
magic = get_ipython().magic  # noqa
magic('%alias_magic -p "-oq -n 750 -r 15" timeobj timeit')
```

Now in Jupyter/Ipython we can use:

```python
timings = %timeobj f()
```

This will save the results of `%timeit -oq -n 500 -r 7 f()` to `timings`.